# Estimació dels paràmetres del model probabilístic p* i q* i sampling

## Context i motivació

Al notebook anterior (`crear_blueprint.ipynb`) es va construir l’estructura base del problema d’alineació de xarxes, definint el wiring (L_wiring), la xarxa observada (A_test) i la funció d’energia (Hamiltoniana) extretes del repositori del paper, així com diferents procediments de permutació per generar configuracions de referència construïdes reordenant l'observació, permutant etiquetes i fent shuffles; en comptes de mostrejar-les a partir d'un model generatiu.

L’objectiu d’aquest notebook és reformular l’anàlisi des d’un punt de vista probabilístic. En lloc de generar configuracions mitjançant shuffles, es treballa amb un ensemble de xarxes generades directament des del model, mantenint fixa l’estructura del wiring (un cop introduïda la probabilitat) i estimant els paràmetres del model a partir de la xarxa observada.

## Objectius:

- Generar una matriu de wiring (L_wiring) aleatòria, assignant la mateixa probabilitat a tots els enllaços possibles.


- Estimar els paràmetres òptims del model p* iq* a partir de (L_wiring) i de la xarxa observada (A_test).

- Generar un ensemble de xarxes utilitzant els paràmetres estimats i la permutació identitat.

- Calcular la distribució d’energies del model i comparar-la amb l’energia observada per tal d’avaluar la significança estructural de l’alineació.

Aquest enfocament permet construir un model nul coherent amb l’estructura del sistema i analitzar quantitativament fins a quin punt la configuració observada és compatible amb el model probabilístic considerat.

In [9]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

## Generació del wiring aleatori

En lloc d’inferir el wiring a partir de les observacions, es considera un wiring aleatori amb probabilitat uniforme per a cada enllaç possible, amb l’objectiu d’evitar biaixos estructurals introduïts per la mostra finita de dades. No mirem dades (ni A_f ni P_inv_f) pq volem un null model.


En l’enfocament anterior, la matriu de wiring  es construïa a partir de les K xarxes observades alineades, assignant un enllaç en cada posició del blueprint si aquest apareixia en la majoria de les observacions.

En aquest notebook s’adopta una estratègia diferent. Amb l’objectiu de definir un model nul més neutre i evitar biaixos estructurals associats a una mostra finita d’observacions, el wiring es genera de manera aleatòria i independent de les xarxes observades. Concretament, cada entrada del wiring es construeix com una variable aleatòria de Bernoulli amb probabilitat uniforme.

Aquest procediment assigna la mateixa probabilitat a tots els enllaços possibles i garanteix que el wiring no codifica cap estructura preferencial derivada de les dades. Un cop generada, la matriu es manté fixa durant tota la resta de l’anàlisi, i s’utilitza tant per a l’estimació dels paràmetres del model com per a la generació de l’ensemble de xarxes.

Aquesta separació clara entre la definició estructural del model (wiring) i la informació empírica (xarxa observada) permet interpretar de manera més rigorosa la distribució d’energies obtinguda i avaluar fins a quin punt la configuració observada és compatible amb les fluctuacions esperades sota el model probabilístic considerat.



#### Dubtes

La matrius és simetrica? és a dir, un link a-->b implica b-->a? (L_ij=L_ji?)
La diagonal és 0? Existeixen self-links Lii?

Atès que la xarxa neuronal de C. elegans es representa mitjançant una matriu d’adjacència dirigida, no s’imposa simetria en la construcció del wiring. En canvi, es força la diagonal a zero, ja que no es consideren autoconnectivitats.
Atès que les dades de connectoma no inclouen autoconnectivitats i que aquestes no estan contemplades en la Hamiltoniana del model, es força la diagonal del wiring a zero

In [11]:
def L_wiring_random(Nx, Ny): 
    L_wiring= np.zeros((Nx,Ny))

    for i in range(0,Nx):
        for j in range(0, Ny):
            if np.random.rand() < 0.5:
                L_wiring[i,j]=1
            else:
                L_wiring[i,j]=0
    #vull imposar que la diagonal sigui 0
    for i in range(0, min(Nx,Ny)):
        L_wiring[i,i]=0
    return L_wiring
Nx=20
Ny=20
L_wiring_nova = L_wiring_random(Nx, Ny)
print(f"L_wiring generada aleatòriament té la forma:{L_wiring_nova.shape}")
#vull imprimir la diagonal per comprovar que és 0
print("Diagonal de L_wiring_nova:") 
print(np.diag(L_wiring_nova))

L_wiring generada aleatòriament té la forma:(20, 20)
Diagonal de L_wiring_nova:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Estimació dels paràmetres p* i q*

##  Generació de l’ensemble de xarxes

## Càlcul de l’energia

## Histograma i comparació amb l’observació

## Conclusions